<a href="https://colab.research.google.com/github/sofiadgelis/ML-AudioEnhancement-Project/blob/main/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---   
# **Audio Restoration for Generative Models — Improving MusicGen Outputs**
---  

**L’obiettivo** di questo progetto, apparentemente semplice, è quello di migliorare un audio generato da MusicGen. In realtà, la sfida principale consiste nell’aumentarne la qualità sia dal punto di vista percettivo sia da quello tecnico, mantenendo inalterata l’accuratezza semantica rispetto al prompt originario.

Nella **prima cella** del notebook è stata predisposta l’inizializzazione dell’ambiente di lavoro mediante l’installazione e l’importazione delle librerie necessarie, e tramite la creazione delle cartelle per il salvataggio degli output del progetto:   
* **torch** e **torchaudio**: la prima costituisce uno strumento fondamentale per il Machine Learning e il Deep Learning, in quanto consente la gestione di tensori e modelli neurali; la seconda è dedicata al processamento e alla manipolazione di segnali audio.  

* **numpy**: utilizzata per il calcolo scientifico, in particolare per la gestione di array e operazioni numeriche.

* **librosa**: fondamentale per analizzare e processare i file audio, la useremo durante l'analisi finale.

* **librosa.display** e **matplotlib**: visualizzano i risultati delle precedenti analisi (spettrogrammi, waveform) creando grafici e diagrammi.

* **os** e **glob**: consentono la gestione e la navigazione di file e directory.

* **transformers**: libreria essenziale per il caricamento e l’utilizzo di modelli pre-addestrati, tra cui MusicGen che noi andiamo ad utilizzare.

* **scipy.io.wavfile (write)**: impiegata per la scrittura di file audio in formato .wav.

* **IPython.display (Audio, display)**: permette la riproduzione e la visualizzazione dei file audio direttamente all’interno di notebook interattivi come Jupyter.





In [ ]:
# installazione librerie, tra cui demucs che andremo a utilizzare per il miglioramento audio
!pip install torch torchaudio numpy librosa scipy soundfile matplotlib transformers accelerate demucs

# import delle librerie
import torch
import torchaudio
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import glob
from transformers import AutoProcessor, MusicgenForConditionalGeneration
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio, display

# creazione delle cartelle di lavoro

os.makedirs('musicgen_output', exist_ok=True) # cartella per audio generato
os.makedirs('processed_audio', exist_ok=True) # cartella per audio processato
os.makedirs('demucs_output', exist_ok=True) # cartella per gli output di Demucs
os.makedirs('enhanced_audio', exist_ok=True) # cartella per audio finale

print("Setup completato. Librerie importate e cartelle create.")

Nella **seconda cella** viene effettuato il caricamento del modello **MusicGen**, sviluppato da Meta AI, un modello di text-to-music generation che è in grado di generare sequenze audio coerenti a partire da un prompt in linguaggio naturale, mantenendo sia la struttura musicale che lo stile richiesto.

In questo progetto, il modello viene inizializzato per leggere il prompt ( **prompt_text**) fornito e generare l’audio corrispondente. Una volta prodotto, il file audio viene salvato all’interno della cartella predefinita.

Volendo nella riga 11, per rendere più dinamico il progetto, si può inserire una funzione:
```
prompt_text = [input('inserisci prompt personalizzato':)]
```
che permette all'utente di definire l'audio che vuole.

In [ ]:
# caricamento MusicGen

print("Caricamento del modello MusicGen...")
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Modello MusicGen caricato su: {device}")

# definizione del prompt
prompt_text = ["An 80s pop song with heavy synth, a groovy bassline, and punchy drums"]

# generazione dell'audio
print("generazione dell'audio in corso...")
inputs = processor(
    text=prompt_text,
    padding=True,
    return_tensors="pt"
).to(device)

# per una durata maggiore aumentiamo i token
audio_values = model.generate(**inputs, max_new_tokens=768)
sampling_rate = model.config.audio_encoder.sampling_rate
print("Generazione completata.")

# salvataggio dell'output
# normalizziamo e convertiamo in 16-bit PCM per simulare un output standard
audio_numpy = audio_values.cpu().numpy().squeeze()
audio_numpy = np.int16(audio_numpy / np.max(np.abs(audio_numpy)) * 32767)

musicgen_raw_path = 'musicgen_output/raw_output_32k_16bit.wav'
write_wav(musicgen_raw_path, sampling_rate, audio_numpy)

print(f"File audio salvato in: {musicgen_raw_path}")
print("Output di MusicGen:")
display(Audio(musicgen_raw_path))

Nella **terza cella** viene eseguita la fase di **pre-processing** del segnale audio, accompagnata dalla conversione di formato mediante operazioni di **Digital Signal Processing (DSP)**. Questa fase ha lo scopo di predisporre il suono agli step successivi di elaborazione: le modifiche effettuate non sono percepibili all’ascolto, ma sono necessarie dal punto di vista tecnico per garantire la compatibilità e la qualità del segnale.

La funzione principale implementa tre operazioni:

* Caricamento dei file audio: i dati vengono letti dal disco e trasferiti in memoria per poter essere elaborati.

* Upsampling: la frequenza di campionamento viene incrementata da 32 kHz a 48 kHz, migliorando la risoluzione temporale del segnale e rendendolo conforme a standard audio più diffusi.

* Conversione di formato: i dati audio vengono trasformati dal formato a interi a 16 bit in numeri in virgola mobile a 32 bit (32-bit float), garantendo una maggiore precisione numerica. Successivamente, i file vengono nuovamente salvati su disco.

In [ ]:
def preprocess_audio_stage1_alternative(input_path, output_path, target_sr=48000):

    print(f"Inizio pre-processing per: {input_path} ")

    waveform, original_sr = torchaudio.load(input_path)
    print(f"File caricato. SR originale: {original_sr} Hz, Dtype: {waveform.dtype}")

    # uso del resampler interno di torchaudio
    resampler = torchaudio.transforms.Resample(
        orig_freq=original_sr,
        new_freq=target_sr,
        resampling_method="sinc_interpolation"
    )
    resampled_waveform = resampler(waveform)
    print(f"Upsampling a {target_sr} Hz completato.")

    # salvataggio in formato 32-bit float
    torchaudio.save(
        output_path,
        resampled_waveform,
        target_sr,
        encoding="PCM_F",
        bits_per_sample=32
    )
    print(f"--- File processato salvato in: {output_path} ---")

processed_path = 'processed_audio/processed_48k_32bit.wav'
# chiamata della nuova funzione
preprocess_audio_stage1_alternative(musicgen_raw_path, processed_path)

print("\nAudio dopo il Pre-Processing (48kHz, 32-bit float):")
display(Audio(processed_path))

Nella **quarta cella** viene eseguita la fase finale del progetto e tratta proprio il miglioramento del segnale audio attraverso **Demucs (Deep Extractor for Music Sources)**, un modello di source separation sviluppato da Meta AI e che permette di scomporre un brano nelle sue componenti fondamentali con un’elevata fedeltà.

La procedura adottata nel progetto segue due fasi principali:

* Separazione: il modello analizza il brano e lo scompone nelle sue tracce costitutive, tipicamente basso, batteria, voce e una traccia “altro” che include gli strumenti rimanenti.

* Ricombinazione: le tracce separate vengono successivamente riunite per ricostruire l’audio completo.

Il principio alla base di questo approccio è che, durante il processo di separazione e ricostruzione, il modello tende a eliminare i disturbi, i rumori di fondo e gli artefatti che non appartengono a nessuna delle componenti strumentali principali. Il risultato finale è un audio tecnicamente più pulito, definito e percettivamente superiore rispetto all’originale.

In [ ]:
import shutil # fornisce la possibilità di supportare la copia e la rimozione

print(" miglioramento audio con Demucs")

demucs_output_dir = "demucs_output"

# esecuzione del modello
print("Esecuzione di Demucs in corso... (separazione completa)")
!demucs "{processed_path}" -o "{demucs_output_dir}"

print("\n--- Elaborazione Demucs completata ---")

# file di output (bass, drums, other, vocals) che vanno ricercati
try:
    # costruzione del percorso di ricerca
    base_name = os.path.basename(processed_path).replace('.wav', '')
    # modello di default è 'htdemucs'
    search_path = os.path.join(demucs_output_dir, "htdemucs", base_name, "*.wav")

    output_stems = glob.glob(search_path)

    if len(output_stems) < 1: # può genera meno di 4 stem se sono vuoti
        raise FileNotFoundError("Nessuna traccia separata trovata. Controlla l'output di Demucs.")

    print(f"Trovate {len(output_stems)} tracce separate:")
    for stem_path in output_stems:
        print(f" - {os.path.basename(stem_path)}")

    # cariacmento e unione tracce
    combined_waveform = None
    final_sr = None

    for stem_path in output_stems:
        waveform, sr = torchaudio.load(stem_path)
        if combined_waveform is None:
            combined_waveform = waveform
            final_sr = sr
        else:
            # controllo che i tensori abbiano la stessa lunghezza
            target_length = min(combined_waveform.shape[1], waveform.shape[1])
            combined_waveform = combined_waveform[:, :target_length] + waveform[:, :target_length]

    # salvataggio file finale
    enhanced_path = os.path.join('enhanced_audio', 'final_enhanced_output_recombined.wav')
    torchaudio.save(enhanced_path, combined_waveform, final_sr)
    print(f"\nTracce ricombinate e salvate in: {enhanced_path}")

    # audio finale
    print("\nAudio dopo il Miglioramento (Finale con Demucs Ricombinato):")
    display(Audio(enhanced_path))

except Exception as e:
    print(f"ERRORE: {e}")

# **Analisi degli output**
---
Nelle **ultime celle**  del notebook l’attenzione viene posta sull’analisi dei risultati ottenuti, attraverso strumenti sia visivi che quantitativi. In particolare, vengono utilizzati:

* lo spettrogramma, che consente una rappresentazione visiva del segnale sonoro e delle sue variazioni nel tempo;

* un grafico a barre che mostra, in termini numerici e percentuali, il miglioramento della qualità audio.

**Quinta cella: calcolo e visualizzazione degli spettrogrammi**  

In questa fase viene calcolato lo spettrogramma per tre diverse versioni del segnale: l’audio originale, quello processato e quello finale.
Lo spettrogramma è una rappresentazione bidimensionale che mostra le frequenze (da quelle più basse a quelle più alte) presenti in un suono e la loro evoluzione temporale. I grafici vengono visualizzati uno sotto l’altro, permettendo così un confronto diretto della “trama” del segnale acustico prima e dopo il miglioramento.

In [ ]:
# la prima funzione serve per il plot dello spettrogramma
def plot_spectrogram(filepath, title, ax):
    y, sr = librosa.load(filepath)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    img = librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel', ax=ax)
    ax.set_title(title)
    return img

# creazione grafici
fig, axs = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# 1. Spettrogramma Originale
plot_spectrogram(musicgen_raw_path, f'1. Originale MusicGen ({torchaudio.info(musicgen_raw_path).sample_rate} Hz)', axs[0])

# 2. Spettrogramma dopo Pre-Processing (Upsampling)
plot_spectrogram(processed_path, f'2. Dopo Upsampling ({torchaudio.info(processed_path).sample_rate} Hz)', axs[1])

# 3. Spettrogramma dopo Miglioramento
img = plot_spectrogram(enhanced_path, f'3. Finale Migliorato ({torchaudio.info(enhanced_path).sample_rate} Hz)', axs[2])

fig.colorbar(img, ax=axs, format='%+2.0f dB', label='Intensità')
plt.tight_layout()
plt.show()

# ascolto audio finale
print("--- Riepilogo Audio ---")
print("\n1. Originale (da MusicGen):")
display(Audio(musicgen_raw_path))

print("\n2. Dopo Upsampling a 48kHz (Stage 1):")
display(Audio(processed_path))

print("\n3. Finale (dopo Miglioramento Neurale):")
display(Audio(enhanced_path))


**Sesta cella: calcolo delle metriche e rappresentazione grafica**

Nell’ultima cella si procede con la valutazione quantitativa del miglioramento attraverso il calcolo di metriche specifiche sia per l’audio originale sia per quello finale:

* Spectral Centroid: indice che misura la “brillantezza” del suono, legato alla concentrazione delle frequenze più alte.

* Spectral Flatness: indice che valuta il grado di “rumorosità” di un segnale; valori più bassi corrispondono a un suono più tonale e meno rumoroso.

* RMS Energy: misura del livello energetico medio, interpretabile come il volume percepito del segnale.

Infine, i risultati vengono rappresentati tramite un grafico a barre che evidenzia la variazione percentuale di ciascun indice, rendendo immediatamente chiaro l’impatto del processo di miglioramento audio.

In [ ]:
def calculate_and_compare_metrics(original_path, enhanced_path):
    print("Analisi Comparativa della Qualità Audio")

    # caricamento file
    y_orig, sr_orig = librosa.load(original_path)
    y_enh, sr_enh = librosa.load(enhanced_path)

    # calcolo metriche

    metrics = {}

    # per l'audio originale
    metrics['original'] = {
        'spectral_centroid': np.mean(librosa.feature.spectral_centroid(y=y_orig, sr=sr_orig)),
        'spectral_flatness': np.mean(librosa.feature.spectral_flatness(y=y_orig)),
        'rms_energy': np.mean(librosa.feature.rms(y=y_orig))
    }

    # per l'audio migliorato
    metrics['enhanced'] = {
        'spectral_centroid': np.mean(librosa.feature.spectral_centroid(y=y_enh, sr=sr_enh)),
        'spectral_flatness': np.mean(librosa.feature.spectral_flatness(y=y_enh)),
        'rms_energy': np.mean(librosa.feature.rms(y=y_enh))
    }

    # risultati

    print("\n" + "="*40)
    print(" RISULTATI NUMERICI")
    print("="*40)
    print(f"{'Metrica':<20} | {'Originale':<15} | {'Migliorato':<15}")
    print("-"*40)

    for key in metrics['original']:
        orig_val = metrics['original'][key]
        enh_val = metrics['enhanced'][key]
        print(f"{key:<20} | {orig_val:<15.4f} | {enh_val:<15.4f}")
    print("="*40 + "\n")

    return metrics

# creazione grafico di confronto
# valori dell'originale posti pari al 100%
def plot_comparison(metrics):
    labels = list(metrics['original'].keys())
    original_values = np.array(list(metrics['original'].values()))
    enhanced_values = np.array(list(metrics['enhanced'].values()))

    # normalizzazione dei valori
    enhanced_normalized = (enhanced_values / original_values) * 100

    x = np.arange(len(labels))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, [100] * len(labels), width, label='Originale (Riferimento 100%)', color='skyblue')
    rects2 = ax.bar(x + width/2, enhanced_normalized, width, label='Migliorato (vs Originale)', color='salmon')

    ax.set_ylabel('Valore Normalizzato (%)')
    ax.set_title('Confronto Qualità Audio: Originale vs. Migliorato')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=15)
    ax.axhline(100, color='grey', linestyle='--', linewidth=0.8)
    ax.legend()

    ax.bar_label(rects2, fmt='%.1f%%', padding=3)

    fig.tight_layout()
    plt.show()


# esecuzione analisi

try:
    comparison_metrics = calculate_and_compare_metrics(musicgen_raw_path, enhanced_path)
    plot_comparison(comparison_metrics)
except NameError:
    print("ERRORE: sono eseguite le celle precedenti? sono le variabili 'musicgen_raw_path' e 'enhanced_path' definite?")